In [2]:
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import ee
import pandas as pd
ee.Initialize()

In [3]:
# set up satellite colleciton of interest
satellite = 'LANDSAT/LC08/C01/T1_SR'

startYear = 2000
endYear = 2020

# set up dates
startDate = '01-01-{}'.format(str(startYear))
endDate = '31-12-{}'.format(str(endYear))

In [4]:
# import shape file for Bangladesh
wd = '/home/beeb/Insync/sei2112@columbia.edu/Google Drive/Columbia SusDev PhD/Research/My work/predictArsenic/'
os.chdir(wd)
# set up band names appropriate for LANDSAT 8
nir = 'B4'
red = 'B3'

In [5]:
countrypath = 'data/raw/shapefiles/bgd_admbnda_adm{level}_bbs_20180410/bgd_admbnda_adm{level}_bbs_20180410.shp'
bgl = gpd.read_file(countrypath.format(level = '0'))
# go from shapefile to a format that GEE can parse
poly = [i.__geo_interface__ for i in bgl.geometry][0]

In [6]:
# function to mask out cloudy pixels
def fixCloud(image):
    cloudScore = ee.Algorithms.Landsat.simpleCloudScore(image)
    mask = cloudScore.select(['cloud']).lte(20);
    masked = image.updateMask(mask);
    return masked

# calculate NDVI
def addNDVI(image):
    ndvi = image.normalizedDifference([nir, red]).rename('NDVI')
    return image.addBands(ndvi)

wholeCollection = ee.ImageCollection(satellite).filterBounds(poly).filterDate(startDate, endDate).map(fixCloud).map(addNDVI)



In [7]:
# this function gets the median NDVI for a given month
def getMedianForMonth(month, collection = wholeCollection):
    print('{year}-{month}-01'.format(month = str(month), year  = startYear))
    # earliest example of that month within the timeframe
    startMonth = pd.to_datetime('{year}-{month}-01'.format(month = str(month), year  = startYear))
    # end one month later
    endMonth = startMonth + pd.DateOffset(months  = 1)
    
    # create a featurecollection of the month for every year
    start = [startMonth + pd.DateOffset(years = i) for i in range(endYear - startYear)]
    end = [endMonth + pd.DateOffset(years = i) for i in range(endYear - startYear)]
    
    # initialise the collection
    monthCollection = collection.filterDate(start[0], end[0])

    for i in range(1, len(start)):
        monthCollection = monthCollection.merge(collection.filterDate(start[i], end[i]))
    
    
    # take the median of NDVI and return
    ndviMedian = monthCollection.select('NDVI').reduce(ee.Reducer.median())
    return ndviMedian

In [8]:
# returns a collection of the median NDVI for each month
def createMonthlyCollection(collection):
    overallCollection = getMedianForMonth(1, collection)
    for i in range(2, 13):
        overallCollection = overallCollection.addBands(getMedianForMonth(i, collection))

    return overallCollection

In [9]:
finalCollection = createMonthlyCollection(wholeCollection)

2000-1-01
2000-2-01
2000-3-01
2000-4-01
2000-5-01
2000-6-01
2000-7-01
2000-8-01
2000-9-01
2000-10-01
2000-11-01
2000-12-01


NameError: name 'Map' is not defined